You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
from api_key_store import ApiKeyStore
s = ApiKeyStore()

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from time import time

- Import from the crewAI libray.

In [4]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [5]:
import os

In [6]:
os.environ["OPENAI_API_KEY"] = s.get_api_key(provider="OPENAI")  # os.environ["API_KEY_OPENAI"]
os.environ["SERPER_API_KEY"] = s.get_api_key(provider="SerpApi")  # os.environ["API_KEY_SERPAPI"]

In [7]:
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [8]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [9]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [10]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [11]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [12]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [13]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [14]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
ts_start = time()
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})
ts_stop = time()
print(f"Elapsed time: {ts_stop-ts_start} sec")   # Elapsed time: 38.442201137542725 sec on papagame

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan: The Rise of Artificial Intelligence

I. Introduction
- Brief overview of artificial intelligence (AI) and its increasing impact on various industries.
- Mention of the latest trends in AI technology and its potential benefits.
- Introduction of key players in the AI industry and their contributions.

II. Key Points
A. Latest Trends in AI
1. Machine learning advancements
2. Natural language processing developments
3. Automation and robotics integration
4. AI ethics and regulation updates

B. Key P

- Display the results of your execution as markdown in the notebook.

In [16]:
from IPython.display import Markdown
Markdown(result)

# The Rise of Artificial Intelligence: Latest Trends and Key Players

## Introduction

Artificial intelligence (AI) has become a prominent force reshaping various industries, with its impact continually growing. The latest trends in AI technology are revolutionizing the way we work and interact with machines, offering potential benefits such as increased efficiency, improved decision-making, and enhanced customer experiences. Key players in the AI industry, including Google DeepMind, IBM Watson, Amazon Web Services (AWS), Microsoft AI, and OpenAI, are driving innovation and pushing the boundaries of what AI can achieve.

## Key Points

### Latest Trends in AI

One of the most significant trends in AI is the rapid advancements in machine learning algorithms. These algorithms enable machines to learn from data, identify patterns, and make predictions without explicit programming. As a result, AI systems can continuously improve their performance and adapt to new information, leading to more accurate and efficient outcomes.

Natural language processing (NLP) is another area experiencing significant advancements in AI technology. NLP allows machines to understand, interpret, and generate human language, enabling applications such as chatbots, language translation, and sentiment analysis. With the development of more sophisticated NLP models, AI systems can communicate more effectively and provide more personalized experiences to users.

AI-powered automation and robotics are transforming industries by streamlining processes, increasing productivity, and reducing human error. From autonomous vehicles to robotic process automation, AI technologies are revolutionizing how tasks are performed, leading to greater efficiency and cost savings for businesses.

As AI technology continues to advance, ethical considerations and regulatory frameworks are becoming increasingly important. Issues such as bias in AI algorithms, data privacy concerns, and the impact of AI on employment are shaping the conversation around responsible AI development. Organizations and policymakers are working to establish guidelines and standards to ensure that AI is developed and deployed in a way that benefits society as a whole.

### Key Players in the AI Industry

Google DeepMind is known for its groundbreaking research in AI, particularly in the fields of deep learning and reinforcement learning. Their work has led to advancements in healthcare, gaming, and natural language understanding, demonstrating the potential of AI to solve complex problems and improve human lives.

IBM Watson is a pioneer in cognitive computing, with a focus on building AI systems that can understand, reason, and learn from data. Watson's applications span various industries, including healthcare, finance, and customer service, showcasing the versatility and power of AI to transform business operations.

Amazon Web Services (AWS) offers a range of AI services and tools that enable businesses to integrate AI capabilities into their applications easily. From speech recognition to image analysis, AWS's AI offerings empower organizations to harness the full potential of AI and drive innovation in their products and services.

Microsoft AI is at the forefront of democratizing AI technology, making it accessible to developers and organizations of all sizes. Their AI solutions, such as Azure Cognitive Services and Microsoft Bot Framework, enable businesses to build intelligent applications and services that enhance customer experiences and drive digital transformation.

OpenAI is a research organization dedicated to advancing AI in a way that benefits humanity. Their work focuses on developing safe and beneficial AI systems that can be used to address global challenges and improve people's lives. By promoting transparency, collaboration, and ethical AI development, OpenAI is shaping the future of AI in a responsible and sustainable manner.

### Noteworthy News in AI

AI is revolutionizing healthcare by enabling more accurate diagnoses, personalized treatment plans, and improved patient outcomes. From medical imaging analysis to drug discovery, AI technologies are transforming the way healthcare is delivered, leading to better quality care and reduced costs for patients and providers.

The rise of cyber threats has made cybersecurity a top priority for organizations worldwide. AI-powered cybersecurity solutions can detect and respond to threats in real-time, helping to prevent data breaches, identify vulnerabilities, and protect sensitive information. By leveraging AI capabilities, businesses can enhance their security posture and safeguard their digital assets from cyber attacks.

As AI technology continues to advance, its impact on the future of work is becoming more apparent. Automation and AI-driven technologies are reshaping job roles, creating new opportunities, and transforming the way we work. While AI has the potential to increase productivity and efficiency in the workplace, it also raises concerns about job displacement, skills training, and workforce readiness in the digital age.

Ethical considerations play a crucial role in AI development, ensuring that AI systems are designed and deployed responsibly. Issues such as bias, transparency, accountability, and data privacy require careful attention to prevent unintended consequences and societal harm. By addressing ethical considerations upfront, organizations can build trust, mitigate risks, and foster positive outcomes from AI technologies.

## Target Audience Analysis

Professionals in the tech industry, business leaders, educators, and individuals interested in AI form the target audience for this blog post. They are keen on staying updated on the latest AI trends, understanding AI applications in different fields, and exploring ethical considerations in AI development. Navigating the complexities of AI technology, staying informed on the latest news, and understanding the implications of AI on society are key pain points for this audience.

## Conclusion

Artificial intelligence is continuously evolving, with the latest trends and advancements shaping the future of technology and innovation. Key players in the AI industry are driving progress and pushing the boundaries of what AI can achieve, while ethical considerations and regulatory updates are shaping the responsible development of AI systems. By staying informed on the latest news and developments in AI, professionals can navigate the complexities of this transformative technology and harness its potential for positive impact. Stay tuned for more insights and updates on artificial intelligence by following our blog for regular content and analysis.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"

topic = "Text to SQL"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).